# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math
from scipy import stats


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
from secrets import IEX_CLOUD_SANDBOX_API_TOKEN
stocks_df = pd.read_csv('sp_500_stocks.csv')
stocks_df.columns


Index(['Ticker'], dtype='object')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

Retrieve price and one-year stock returns

In [10]:
base_url="https://sandbox.iexapis.com"
params={"token":IEX_CLOUD_SANDBOX_API_TOKEN}
response = requests.get(f"{base_url}/stable/stock/fb/stats", params=params)
response.json()
# "GET /stock/{symbol}/stats/{stat?}"
# find price and one year stock returns in batch, lets do simple fisrt


{'companyName': 'Facebook Inc',
 'marketcap': 946531744371,
 'week52high': 340.2,
 'week52low': 201.86,
 'week52highSplitAdjustOnly': 347.32,
 'week52lowSplitAdjustOnly': 210.61,
 'week52change': 0.5043959521420666,
 'sharesOutstanding': 2482330304,
 'float': 0,
 'avg10Volume': 19076703,
 'avg30Volume': 19740140,
 'day200MovingAvg': 285.47,
 'day50MovingAvg': 315.2,
 'employees': 52838,
 'ttmEPS': 11.91,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '0',
 'exDividendDate': '0',
 'nextEarningsDate': '2021-07-14',
 'peRatio': 28.095413168946045,
 'beta': 1.2560037460406996,
 'maxChangePercent': 7.461107248906546,
 'year5ChangePercent': 1.696872085767828,
 'year2ChangePercent': 0.7009037398579866,
 'year1ChangePercent': 0.5252513251901025,
 'ytdChangePercent': 0.1686816689589407,
 'month6ChangePercent': 0.14495070072295396,
 'month3ChangePercent': 0.19002784916805004,
 'month1ChangePercent': 0.021227224211726275,
 'day30ChangePercent': 0.022142018668580933,
 'day5Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [22]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

    
symbol_groups = list(chunks(stocks_df['Ticker'], 100))
symbol_strings = []


for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
   
    
#     /stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5
    print(symbol_strings[i]) #string batches
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']


Ticker                        A
Year 1 Price Change    0.711552
dtype: object
Ticker                      AAL
Year 1 Price Change    1.174265
dtype: object
Ticker                      AAP
Year 1 Price Change    0.648774
dtype: object
Ticker                     AAPL
Year 1 Price Change    0.694003
dtype: object
Ticker                     ABBV
Year 1 Price Change    0.460142
dtype: object
Ticker                      ABC
Year 1 Price Change    0.400439
dtype: object
Ticker                     ABMD
Year 1 Price Change    0.593936
dtype: object
Ticker                      ABT
Year 1 Price Change    0.292602
dtype: object
Ticker                      ACN
Year 1 Price Change    0.594792
dtype: object
Ticker                     ADBE
Year 1 Price Change    0.340906
dtype: object
Ticker                      ADI
Year 1 Price Change    0.472666
dtype: object
Ticker                      ADM
Year 1 Price Change    0.943119
dtype: object
Ticker                      ADP
Year 1 Price Change    0.365906


TypeError: string indices must be integers

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [85]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_groups:
    symbol_string = ",".join(symbol_string)
#     print(symbol_string)
    params={"symbols":symbol_string, "types":"stats,price", "token":IEX_CLOUD_SANDBOX_API_TOKEN}
    response = requests.get(f"{base_url}/stable/stock/market/batch", params=params)
   
    data = response.json()
    for ticker in symbol_string.split(','):
        row = pd.Series([
            ticker,
            data[ticker]['price'],
            data[ticker]['stats']['year1ChangePercent'],
            "N/A"
            ], index=my_columns)
        final_df = final_df.append(row, ignore_index=True)

        
#         print(final_df.loc[final_df['Ticker'] == ticker].iloc[0]['One-Year Price Return'])
        one_year_pr = final_df.loc[final_df['Ticker'] == ticker].iloc[0]['One-Year Price Return']
        assert(one_year_pr == data[ticker]['stats']['year1ChangePercent'])

        



Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  134.93              0.721116                     N/A
1      AAL   23.00              1.190223                     N/A
2      AAP  210.11              0.659315                     N/A
3     AAPL  129.77                0.7207                     N/A
4     ABBV  116.47              0.455808                     N/A
..     ...     ...                   ...                     ...
500    YUM  125.54              0.451722                     N/A
501    ZBH  177.91              0.435876                     N/A
502   ZBRA  500.00              1.056317                     N/A
503   ZION   60.20              0.989346                     N/A
504    ZTS  174.80              0.373887                     N/A

[505 rows x 4 columns]

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [ ]:
final_df

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: